In [45]:
import geopandas as gpd
import json

!pwd

/Users/chadbustard/Desktop/GaussianProcess/data


In [46]:
f = open("noisemeters.json")
meters = json.load(f)
gdf = gpd.GeoDataFrame.from_features(meters["features"])
print(gdf.head())



                     geometry
0  POINT (-89.42405 43.05694)
1  POINT (-89.36182 43.08410)
2  POINT (-89.35310 43.09027)
3  POINT (-89.35316 43.09532)
4  POINT (-89.34832 43.09614)


In [54]:
df = gdf.copy()
df['x'] = df['geometry'].x
df['y'] = df['geometry'].y
df

,geometry,x,y
0,POINT (-89.42405 43.05694),-89.424045,43.056938
1,POINT (-89.36182 43.08410),-89.361818,43.084104
2,POINT (-89.35310 43.09027),-89.353100,43.090265
3,POINT (-89.35316 43.09532),-89.353156,43.095317
4,POINT (-89.34832 43.09614),-89.348319,43.096139
5,POINT (-89.34281 43.09191),-89.342807,43.091908
6,POINT (-89.35091 43.10608),-89.350907,43.106077
7,POINT (-89.34562 43.10837),-89.345620,43.108373
8,POINT (-89.34303 43.10603),-89.343032,43.106032
9,POINT (-89.33521 43.11309),-89.335214,43.113095


In [55]:
#check this later
df['location'] = ['Location 02','Location 03','Location 00', 'Location 16', 'Location 15',
                  'Location 08', 'Location 11', 'Location 09', 'Location 17', 'Location 04',
                   'Location 01', 'Location 05', 'Location 13', 'Location 12', 'Location 10',
                    'Location 06', 'Location 14']

In [61]:
df['noise'] = [94.0, 78.0, 90.0, 90.0, 97.0, 82.0, 63.0, 95.0, 106.0, 102.0, 98.0, 96.0, 95.0, 84.0, 97.0, 73.0, 106.0]
df['units'] = ['dB']*17

final_df = df[['noise', 'x', 'y','units']]
final_df.to_csv("transformed_max_noise_Jan11_2024.csv")

In [57]:
import pandas as pd
import numpy as np
noise_readings = pd.read_csv("MaxNoiseJan112024.csv")
for col in df['location']:
    # noise_readings[col] = noise_readings[col].str.split(' ', expand=False)
    noise_readings[col] = noise_readings[col].fillna(0.0)

for col in df['location']:
    df['noise'] = noise_readings[col].max()

print(df['noise'])




0     106.0
1     106.0
2     106.0
3     106.0
4     106.0
5     106.0
6     106.0
7     106.0
8     106.0
9     106.0
10    106.0
11    106.0
12    106.0
13    106.0
14    106.0
15    106.0
16    106.0
Name: noise, dtype: float64


                      geometry          x          y     location
0   POINT (-89.42405 43.05694) -89.424045  43.056938  Location 02
1   POINT (-89.36182 43.08410) -89.361818  43.084104  Location 03
2   POINT (-89.35310 43.09027) -89.353100  43.090265  Location 00
3   POINT (-89.35316 43.09532) -89.353156  43.095317  Location 16
4   POINT (-89.34832 43.09614) -89.348319  43.096139  Location 15
5   POINT (-89.34281 43.09191) -89.342807  43.091908  Location 08
6   POINT (-89.35091 43.10608) -89.350907  43.106077  Location 11
7   POINT (-89.34562 43.10837) -89.345620  43.108373  Location 09
8   POINT (-89.34303 43.10603) -89.343032  43.106032  Location 17
9   POINT (-89.33521 43.11309) -89.335214  43.113095  Location 04
10  POINT (-89.33088 43.10899) -89.330883  43.108989  Location 01
11  POINT (-89.32920 43.10538) -89.329196  43.105375  Location 05
12  POINT (-89.32891 43.11872) -89.328915  43.118720  Location 13
13  POINT (-89.36330 43.14591) -89.363302  43.145911  Location 12
14  POINT 

In [20]:
import folium
from folium.plugins import HeatMap

# Create a base map
min_lat, max_lat = 42, 44
min_lon, max_lon = -90, -88

m = folium.Map(
    max_bounds=True,
    location=[43.0722, -89.401],
    zoom_start=12,
    min_lat=min_lat,
    max_lat=max_lat,
    min_lon=min_lon,
    max_lon=max_lon,
    tiles = None
)

folium.raster_layers.TileLayer(tiles='cartodbpositron', name='Madison, WI').add_to(m)

# Add Zipcar locations
# separate lat and lon from geometry
markers = folium.FeatureGroup(name="Noise Meter Locations",control=True,show=True).add_to(m)
for index, row in df.iterrows():
    folium.Marker([row['y'], row['x']]).add_to(markers)

m.add_child(folium.LayerControl())
m

